In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

In [ ]:
train = pd.read_csv(u'data/c_13_houseprice.csv')
train.head(3)

In [ ]:
import scipy.stats as st
import seaborn as sns

plt.figure()
sns.distplot(train['SalePrice'])

另一种查看是否服从正态分布的可视化方法


In [ ]:
plt.figure()
res = st.probplot(train['SalePrice'], plot=plt)

把房价做对数变换后再看

In [ ]:
SalePrice_log = np.log(train['SalePrice'])
 
#transformed histogram and normal probability plot
sns.distplot(SalePrice_log, fit=st.norm);
plt.figure()

 

In [ ]:
res = st.probplot(SalePrice_log, plot=plt)